# Notebook 10: Logistische Regression und erweiterte Performance Masse

## Lernziele

* Idee und Modell der Logistischen Regression verstehen.
* Konzept von Fehler bzw. Cost-Functions und Fehler-/Cost-Function der Logistischen Regression.
* Prognose von Klassenzugehörigkeit mittels Threshold.
* Prognsotizieren von Wahrscheinlichkeiten und Klassenzugheörigkeit.
* Confusion-Matrix, Vorhersagegenauigkeit, True Positive Rate und False Positive Rate.
* ROC-Kurve und AUC.

# Das logistische Regressionsmodell

In diesem Notebook lernen wir einen neuen Machine Learning Algorithmus für Klassifikationsprobleme kennen: die **Logistische Regression**.

Obwohl der Name das Wort 'Regression' beinhaltet, ist die Logistische Regression ein bei **Klassifikationsproblemen** verwendetes Modell. Anders als das kNN-Modell, welches **direkt** die Klassenzugehörigkeit vorhersagt, liefert die Logistische Regression aber zunächst eine Vorhersage der **Wahrscheinlichkeit einer bestimmten Klassenzugehörigkeit**.

Um die Logistische Regression kennen zu lernen, verwenden wir hier einen (realen) Datensatz, der für verschiedene Unternehmen als Features einige wichtige Finanzkennzahlen enthält ('Accounting Ratios' wie z.B. das Dept-to-Equity Ratio, im folgenden werden wir diese Kennzahlen schlicht 'ratio' nennen). Als Label haben wir zu jedem Unternehmen die Information, ob es zahlungsunfähig wurde ('Default') oder nicht ('non-Default'). Aufgabe des Machine Learning Algorithmus Logistische Regression wird es also sein, auf Basis der Angaben der Finanzkennzahlen ('ratios') eines Unternehmens die Wahrscheinlichkeit eines Defaults vorauszusagen, bzw., daraus abgeleitet, ob das Unternehmen zu der Klasse der Unternehmen gehört, die zahlungsunfähig werden (Klasse 'Default'), oder zur Klasse derer, die ihren Kredit ordnungsgemäss zurückzahlen werden (Klasse 'non-Default').

**Grundidee:**

Die Wahrschienlichkeit eines Unternehmens, zahlungsunfähig zu werden hängt von den Ratios ab, z.B. dem Debt-to-Equity Ratio. Bei diesem Ratio erwarten wir folgendes Verhalten: Ist das Ratio sehr klein (d.h im Verhältnis zum Eigenkapital sehr wenig Schulden), so ist die Wahrscheinlichkeit eines Defaults sehr klein. Ist das Ratio sehr gross (sehr viele Schulden) so ist die Wahrscheinlichkeit eines Defaults sehr gross. Um die *Ausfallwahrscheinlichkeit* (Wahrscheinlichkeit eines Defaults) $p(x)$ in Abhängigkeit eines solchen Ratios $x$ zu modellieren, benötigen wir also eine Funktion, die für sehr kleine $x$ einen Wert nahe 0 annimmt und dann für grösser werdende $x$ (in kontinuierlicher Weise) auf einen Wert nahe 1 ansteigt. Die Logistische Funktion
$$
y = p(x) = \frac{1}{1+e^{-(\beta_0 + \beta_1 x)}}
$$
zeigt genau ein solches Verhalten.

![LogFunct](LogFunct.png)

**Bemerkung:**

Die logistische Funktion hängt (im einfachsten Fall) ab von den Features $x$ und den **Modellparametern** $\beta_0$ und $\beta_1$. Durch diese beiden Parameter wird die Position (durch $\beta_0$) und die Steilheit (durch $\beta_1$) des Übergangs von $p(x)\approx 0$ auf $p(x)\approx 1$ bestimmt.

Zunächst wollen wir die Form der logistischen Funktion bei verschiedenen Parametern $\beta_0$ und $\beta_1$ etwas besser kennenlernen.

In [ ]:
# Vorbereitung: wir werden numpy und seaborn benötigen
import numpy as np
import pandas as pd
import seaborn as sns

Im Folgenden erstellen wir zunächst einen Vektor *x_werte*, der 200 Werte von -10 bis +10 in Schritten von 0.1 enthält. Dafür benutzen wir die numpy Funktion *arange()*.

In [ ]:
# wir definieren zunächst eine Reihe von x-Werten, an denen wir die logistische Funktion auswerten wollen
x_werte = np.arange(-10,10,0.1)
x_werte

Im nächsten Schritt legen wir die Parameter der logistischen Funktion in den Variablen *beta_0* und *beta_1* fest. Wir könnten hier im Prinzip beliebige reelle Zahlen einsetzen, benutzen aber zunächst mal die einfachsten, nämlich $\beta_0 = 0$ und $\beta_1 = 1$.

In [ ]:
# Parameter der logistischen Funktion festlegen.
# Bemerkung: Diese Werte werden sie später im Rahmen des Aufgabenblocks 1 verändern.
beta_0 = 0
beta_1 = 1

Mit Hilfe der numpy *Vektor Operationen* können wir nun ganz einfach **in einem Schritt** für alle Elemente des Vektors *x_werte* die entsprechenden Funktionswerte berechnen.

In [ ]:
# Funktionswerte berechnen (Beachten Sie, da x_werte ein Vektor ist, ist auch f_x ein Vektor!)
f_x = 1/(1+np.exp(-(beta_0 + beta_1*x_werte)))

In [ ]:
# Schauen Sie sich den Vektor f_x an. Was sind die ersten und die letzten Werte? Machen die Sinn?
f_x

Aus der Tabelle der Werte von *f_x* erkennen wir noch nicht sehr viel. Viel anschaulicher ist ein Plot der Funktion. Wir kennen schon einige Plot-Funktionen aus seaborn. Da wir die Funktion als eine "Linie" darstellen möchten, eignet sich hier der seaborn *lineplot()*. Wir übergeben der Funktion *lineplot()* als x-Koordinaten die Werte des Vektors *x_werte* und als y-Koordinaten die Werte des Vektors *f_x*. Details der Möglichkeiten und Parameter der Funktion *lineplot()* siehe: https://seaborn.pydata.org/generated/seaborn.lineplot.html

In [ ]:
# Funktion darstellen: das geht mit einem seaborn lineplot
sns.lineplot(x = x_werte, y = f_x)

**Interpretation/Beobachtung**: Der Parameter $\beta_0$ bestimmt die Position des Übergangs. Der Parameter $\beta_1$ die Steilheit des Übergangs. 

### Exkurs: Die Matplotlib Plot Funktion

Die seaborn Funktion *lineplot()* ist sehr mächtig und eigentlich für so einen einfachen Plot wie oben ein bischen ein "overkill". Für solche *einfachen* Plots ist es manchmal fast einfacher, diese direkt mit den Plot Funktionen des Basis-Pakets **matplotlib.pyplot** zu erstellen. Der matplotlib Funktion *plot()* können dabei als erste beiden Argumente die (Listen der) x-Werte und y-Werte der zu plottenden Punkte übergeben werden. Einige mögliche Optionen zur Verschönerung der Plots sind *color=*, *linestyle=*, *linewidth=*, *marker=* und *markersize=*. Um Genaueres über diese optionalen Parameter zu erfahren, siehe z.B. https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html unter \**kwargs.

In [ ]:
# Die matplotlib plot Funktionen importieren. Sind ab hier mit dem Prefix plt. ansprechbar.
import matplotlib.pyplot as plt

In [ ]:
# Die matplotlib.plt Funktion
plt.plot(x_werte, f_x)

### Exkurs: Ende

## Lernprozess der Logistischen Regression und Cross-Entropy

Bei der Logistischen Regression ist es Ziel des **Lernprozesses** (Fit des Modells an die Daten), bei **gegebenen Features** $x$ der Unternehmen im Datensatz die **Parameter** $\beta_0$ und $\beta_1$ der Logistischen Funktion so zu **bestimmen**, dass die Funktion $p(x; \beta_0,\beta_1)$ für **alle** Unternehmen im Sample die **wahren Ausfallwahrscheinlichkeiten so gut als möglich reproduziert**.

**Problem:** Die wahren Ausfallwahrscheinlichkeiten der Unternehmen im Datensatz kennen wir gar nicht! Wir wissen nur, ob das Unternehmen tatsächlich ausgefallen ist oder nicht.

Die Lösung ist, ein geeignetes **Fehlermass** für die Vorhersagen des Modell zu definieren. Wir brauchen also ein Mass, das uns sagt, wann eine vorhergesagte Ausfallwahrscheinlichkeit gut ist, und wann sie schlecht ist.
Für ein Unternehmen $i$ mit Feature $x_i$, welches **tatsächlich zahlungsunfähig** geworden ist, sollte der **Fehler gross** sein, wenn das Modell *fälschlicherweise* eine **kleine Ausfallwahrscheinlichkeit** $p_i = p(x_i; \beta_0, \beta_1)$ vorausgesagt hat. Umgekehrt sollte der **Fehler klein** sein, wenn das Modell *richtigerweise* eine **grosse Ausfallwahrscheinlichkeit** vorausgesagt hat. Genau diese Eigenschaft hat die folgende **Fehlerfunktion** $J_i(\beta_0, \beta_1)$, die die Grösse des Fehlers angibt:

$$
J_i(\beta_0, \beta_1) = -\log(p_i) 
$$

Wir sehen: Für $p_i \rightarrow 0$ geht $\log(p_i)\rightarrow -\infty$, d.h. bei kleinen vorausgesagten Ausfallwahrscheinlichkeiten $p_i$ wird der Fehler $J_i$ sehr gross (wir betrachten hier ja ein Unternehmen, das tatsächlich zahlungsunfähig wurde). Umgekehrt gilt für $p_i\rightarrow 1$, dass $\log(p_i)\rightarrow 0$; d.h. für grosse vorausgesagte Ausfallwahrscheinlichkeiten $p_i$ (nahe 1) wird der Fehler $J_i$ verschwindend klein (da es ja richtig ist, für ein Unternehmen, das tatsächlich zahlungsunfähig wurde, eine hohe Ausfallwahrscheinlichkeit vorauszusagen).

Wir schauen uns nun die Funktion $J(p)$ für verschiedene Werte von $p$ in einem Plot an (so wie wir das oben für die Logistische Funktion selbst gemacht haben).

In [ ]:
# Liste der p-Werte zwischen 0 und 1 generieren
p_werte = np.arange(0.01, 0.99, 0.01)

In [ ]:
# Liste der Funktionswerte J(p) berechnen
J_p = -np.log(p_werte)

In [ ]:
# Funktion j(p) darstellen
sns.lineplot(x = p_werte, y = J_p)
plt.title('Fehlerfunktion für ein Unternehmen, das zahlungsunfähig geworden ist')
plt.xlabel('Vorausgesagte Ausfallwahrscheinlichkeit')
plt.ylabel('Resultierender Fehler J(p)')

Im Datensatz hat es aber nicht nur Unternehmen, die tatsächlich zahlungsunfähig geworden sind, sondern auch viele, die **nicht zahlungsunfähig** geworden sind. Für ein Unternehmen, das in Wahrheit **nicht zahlungsunfähig** geworden ist, müsste die Fehlerfunktion genau **umgekehrt** wie oben sein, nämlich so aussehen:

$$
J_i(\beta_0, \beta_1) = -\log(1-p_i)
$$

Schauen wir uns auch diese Funktion in einem Plot an:

In [ ]:
p_werte = np.arange(0.01, 0.99, 0.01)   # Vektor der x-Werte
J_p_nd = -np.log(1-p_werte)                # Vektor der Funktionswerte J(p) 
sns.lineplot(x = p_werte, y = J_p_nd)
plt.title('Fehlerfunktion für ein Unternehmen, das NICHT zahlungsunfähig geworden ist')
plt.xlabel('Vorausgesagte Ausfallwahrscheinlichkeit')
plt.ylabel('Resultierender Fehler J(p)')

Der **Gesamtfehler** des Modells ergibt sich dann durch Summation der Vorhersagefehler jedes einzelnen Unternehmens und kann folgendermassen geschrieben werden:

$$
J(\beta_0, \beta_1) = -\sum_{i=1}^n\{ y_i\log(p_i) + (1-y_i)\log(1-p_i)\}
$$

wobei $y_i$ der Label des Unternehmens $i$ ist. Also $y_i = 1$, falls das Unternehmen $i$ tatsächlich zahlungsunfähig geworden ist, und sonst 0.

**Bemerkung**: In der Summation über alle Unternehmen wird bei jedem Summanden jeweils nur ein Term "behalten". Für ein Unternehmen *i*, das tatsächlich ausgefallen ist, gilt $y_i=1$. Dann wird der erste Term $y_i\log(p_i)$ behalten, aber der zweite Term, $(1-y_i)\log(1-p_i)$ wird wegen dem Faktor $(1-y_i)$ gleich Null. Für ein Unternehmen, das nicht ausgefallen ist, mit $y_i=0$ ist es genau umgekehrt.

Im Rahmen des Machine Learning wird eine solche Fehlerfunktion $J(\beta_0, \beta_1)$ auch **Cost-Function** genannt. Die soeben definierte Funktion $J(\beta_0, \beta_1)$ wird **Cross-Entropy** genannt. Sie wird üblicherweise bei der Logistischen Regression als Cost-Function genommen.

## Minimierung der Cost-Function

Wir haben oben gesehen, dass die Logistische Funktion die **Parameter** $\beta_0$ und $\beta_1$ enthält, welche **so** zu **bestimmen** sind, **dass** - für den gegebenen Datensatz - der **Fehler** $J(\beta_0, \beta_1)$ **minimiert** wird. Manchmal kann man ein solches Minimum *analytisch* finden durch Nullsetzen der Ableitungen. Im Machine Learning, insbesondere wenn man es mit sehr vielen Features oder komplexeren Modellen zu tun hat, ist es aber fast üblicher, dieses Minimum mit Hilfe *numerischer Verfahren* zu finden.

Für uns sind die mathematisch/numerischen Details der Prozedur, wie das Minimum gefunden wird hier irrelevant. Hauptsache die optimalen Parameter $\beta_0$ und $\beta_1$ werden gefunden, die die Cost-Function minimieren.

## Von **Wahrscheinlichkeiten** zu **Klassenzugehörigkeiten**

Haben wir durch Minimieren der Cost-Function die optimalen Parameter $\beta_0$ und $\beta_1$ gefunden, so liefert die resultierende Logistische Funktion

$$
p(x) = \frac{1}{1+e^{-(\beta_0 + \beta_1 x)}}
$$

für jeden Feature-Vektor $x$ die Wahrscheinlichkeit eines Defaults $p(x)$. Wie kommen wir nun von **Wahrscheinlichkeiten** zu **Klassenzugehörigkeiten**, d.h. zu einer eindeutigen Voraussage: das Unternehmen zahlungsunfähig, ja oder nein?

**Vorgehen**:
* Man definiert eine **Threshold** Ausfallwahrscheinlichkeit $p_{thr}$.
* Ist die vorausgesagte Ausfallwahrscheinlichkeit $p(x)$ grösser als der Threshold, wird das Unternehmen der Klasse *Default* zugeordnet, ansonsten der Klasse *Non-Default*.
* Der Standarwert für den Threshold ist $p_{thr} = 0.5$.

## Anwendungsbeispiel

Wir wollen die Logistische Regression am oben beschriebenen Datensatz mit Unternehmensdaten ausprobieren.

In [ ]:
# Der Datensatz heisst 'DefaultData.csv'
daten = pd.read_csv('DefaultData.csv', index_col = 0)
daten.head()

Die normlerweise notwendige Analyse/Visualisierung der Daten überspringen wir hier grossteils. Nur einige elementare Charakteristika:

In [ ]:
# Anzahl Unternehmen im Datensatz
n_samples = len(daten)
n_samples

In [ ]:
# Anzahl zahlungsunfähiger Unternehmen
n_defaults = daten['status'].sum()
n_defaults

In [ ]:
# Anteil zahlungsunfähiger Unternehmen
n_defaults / n_samples

Im Folgenden verwenden wir **alle** Ratios als Features und das Feld *status* als Label.

In [ ]:
# Feature Matrix und Labels
X = daten[['ratio1', 'ratio2', 'ratio3', 'ratio4', 'ratio5']].values
y = daten['status'].values

In [ ]:
# Zur Kontrolle: die Feature-Matrix
X

In [ ]:
# Zur Kontrolle: der Vektor der Labels
y

## Analyse mittels Logistischer Regression

In scikit-learn gibt es für die Logistische Regression die Funktion *LogisticRegression()*.

In [ ]:
# Modell Logistische Regression verfügbar machen
from sklearn.linear_model import LogisticRegression

In [ ]:
# Modell definieren
model = LogisticRegression(solver = 'lbfgs', penalty = None)
model.get_params()

Die scikit-learn Funktion *LogisticRegression()* hat etliche Parameter. Für uns wichtig sind lediglich die Parameter:

* *penalty = None*: Regularisierungsparameter: wir verwenden hier *keine* Regularisierung.
* *solver = ...*: Der zu verwendende Algorithmus für die numerische Minimierung der Cost-Function. Wir belassen das auf dem Default Wert 'lbfgs'.

In [ ]:
# Das Modell (auf dem gesamten Datensatz) trainieren
model.fit(X,y)

Durch Aufruf der Methode *fit()* werden die optimalen Parameter $\beta_0$ und $\beta_1$ der Logistischen Funktion durch Minimieren der oben beschriebenen Cross-Entropy Cost-Function gefunden. Bemerkung: da wir hier 4 Features verwenden, gibt es insgesamt 5 Parameter, nämlich: $\beta_0, \beta_1, \beta_2, \beta_3, \beta_4$ und $\beta_5$.

In [ ]:
# So kann man sich die ("erlernten") Parameter anschauen.
model.coef_

Mit dem trainierten Logistischen Modell kann man nun Vorhersagen für die **Wahrscheinlichkeit** eines Defaults eines Unternehmens machen. Die scikit-learn Methode für die Vorhersage dieser Wahrscheinlichkeit ist *predict_proba()*. Als Argument müssen wir eine Feature-Matrix übergeben. Die Funktion liefert dann für jede Zeile in der Feature-Matrix für jede mögliche Klasse die Wahrscheinlichkeit zu dieser Klasse zu gehören. In unserem Fall gibt es genau zwei Klassen, nämlich die Klasse 0 (non-Default) und die Klasse 1 (Default).

In [ ]:
# In Sample Voraussagen der non-Default- (erste Spalte) und Default- (zweite Spalte) Wahrscheinlichkeiten
# für die ersten 10 Unternehmen
model.predict_proba(X[:10])
# Bemerkung: da es nur die Klassen 0 und 1 gibt, muss die Wahrscheinlichkeit, zur Klasse 1 zugehören,
# genau 1 - die Wahrscheinlichkeit, zur Klasse 0 zu gehören, sein.

In [ ]:
# Eigentlich benötigen wir nur die vorausgesagte Default-Wahrscheinlichkeiten:
# Wir behalten im von der Methode predict_proba() gelieferten Array NUR DIE ZWEITE Spalte
y_pred_proba = model.predict_proba(X[:10])[:,1]
y_pred_proba

Die Logistische Regression sagt also für das erste Unternehmen im Datensatz eine Ausfallwahrscheinlichkeit von 45% voraus, während die vorausgesagte Ausfallwahrscheinlichkeit für das zweite Unternehmen nur 1% beträgt.

Wir schauen uns im Folgenden die für den gesamten Datesatz prognostizierten Ausfallwahrscheinlichkeiten genauer an.

In [ ]:
# Vektor der Ausfallwahrscheinlichkeiten aller Unternehmen im Datensatz
y_pred_proba = model.predict_proba(X)[:,1]
y_pred_proba

In [ ]:
# Visualisierung der Verteilung der prognostizierten Ausfallwahrscheinlichkeiten
sns.histplot(y_pred_proba)

Wenn wir entscheiden müssen üder die Vergabe eines Kredites an ein Unternehmen, interessiert uns eigentlich einfach, ob das Unternehmen den Kredit ordnungsgemäss zurückzahlen wird oder zahlungsunfähig wird; d.h. wir wollen das Unternehmen einer von zwei Klassen zuordnen:

* Klasse 0: Unternehmen wird **nicht** zahlungsunfähig (zahlt Kredit ordnungsgemäss zurück)
* Klasse 1: Unternehmen wird zahlungsunfähig (zahlt den Kredit **nicht** zurück)

Aus den vom Logistischen Modell vorhergesagten **Ausfallwahrscheinlichkeiten** kann man nun die Klassenzugehörigkeit ableiten, indem man einen **Threshold** $p_{thr}$ definiert, ab dem das Unternehmen als "wird zahlungsunfähig" eingestuft wird. Mit einem Threshold von $p_{thr} =0.5$ erhalten wir folgende vorausgesagte Klassenzugehörigkeiten:

In [ ]:
y_pred = (y_pred_proba > 0.5)*1
y_pred

**Einfacher** erhält man die vorhergesagte Klassenzugehörigkeit bei einem Threshold von $p_{thr} = 0.5$ **direkt** mit der scikit-learn Methode *predict()*.

In [ ]:
# Klassenzugehörigkeit bei Threshold 0.5:
model.predict(X[:10])

Vergleichen wir diese Voraussagen mit der **wahren** Klassenzugehörigkeit:

In [ ]:
y[:10]

Unsere vorausgesagten Klassenzugehörigkeiten waren für diese ersten 10 Unternehmen also nicht sehr gut. Das Machine Learning Modell hat keine der beiden Ausfälle erkannt.

**Beobachtung**: Durch Verändern des Thresholds können wir die vorausgesagten Klassenzugehörigkeiten beeinflussen. Mit einem tieferen Threshold werden wir mehr Ausfälle prognsotizieren, mit einem höheren Threshold weniger. Wir probieren also verschiedene Thresholds aus und schauen uns die resultierenden Prognosen an.

Wenn wir andere Thresholds als $p_{thr} = 0.5$ verwenden wollen, so müssen wir die Klassifizierung **"von Hand"** vornehmen.

In [ ]:
# So erhalten wir die prognostizierten Klassenzugehörigkeiten bei einem Threshold p_thr = 0.3
y_pred = (y_pred_proba > 0.3) * 1
y_pred

In [ ]:
# Vergleich der Anzahl vorhergesagter Defaults mit den tatsächlichen:
print('Anzahl vorausgesagter Defaults:', y_pred.sum())
print('Anzahl tatsächlicher Defaults:', y.sum())

In [ ]:
# Anzahl korrekter Vorhersagen
(y_pred == y).sum()

In [ ]:
# Anteil korrekter Vorhersagen (bei Threshold p_thr = 0.3)
(y_pred == y).mean()

In [ ]:
# ACHTUNG: mit der score() Methode erhalten wir den Anteil korrekter Vorhersagen bei einem Threshold von p_thr = 0.5!!!
model.score(X,y)

# Diskussion der Modell-Performance

Im Folgenden wollen wir die Performance des Modells genauer anschauen.

## Benchmark: Der *naive* Schätzer/ Das *naive* Modell

Ob sich der Aufwand eines (mehr oder weniger) komplizierten Machine Learning Modells lohnt, ergibt sich durch **Vergleich mit einem einfacheren Modell**. Das einfachst mögliche Modell ist der sogenannte **Naive Schätzer**. Dieser sagt bei Klassifikationsproblemen **für alle** Datenpunkte **immer** die Klassenzugehörigkeit voraus, die im Sample am häufigsten vorkommt. In unserem Fall ist die Häufigste Klasse der *Non-Default* (fast 90% der Unternehmen werden nicht zahlungsunfähig); d.h. der naive Schätzer würde in unserem Fall für **alle** Unternehmen voraussagen, dass sie **nicht** zahlungsunfähig werden.

Was wäre der (Genauigkeits-)Score des Naiven-Schätzers?

In [ ]:
# Zunächst die Voraussagen des naiven Schätzers (ein Vektor mit alles Nullen)
y_pred_naiv = 0*y
y_pred_naiv

In [ ]:
# Vergleich der Voraussagen mit der wahren Klassenzugehörigkeit
(y_pred_naiv == y).mean()

Diese ganz simple, "naive" Modell, hat also bereits einen score von fast 90%!

## Andere Performance Masse: *True Positive Rate* und *False Positive Rate*

Die Genauigkeit ist in vielen Fällen (so auch in diesem) nicht unbedingt das beste Mass, die Qualtiät eines Algorithmus zu beurteilen.

In unserem Beispiel kann der Schaden gewaltig sein, falls man einen Kredit an ein Unternehmen vergibt, das später zahlungsunfähig wird. Man verliert in dem Fall den gesamten Kreditbetrag! Andererseits verliert man nur den Zinsertrag (bzw. die Zinsmarge) als Einkommen, wenn man einem Unternehmen, das in der Lage wäre, den Kredit korrekt zurück zu zahlen, den Kredit verweigert. Dieser Zinsertrag oder die Zinsmarge ist aber üblicherweise nur ein Bruchteil des gesamten Kreditbetrags.

Schlussfolgerung: in diesem Beispiel wiegen Fehler "Kredit gegeben, Unternehmen zahlt nicht zurück" also viel schwerer als Fehler "kein Kredit gegeben, Unternehmen hätte aber zurückgezahlt". Bei der Berechnung der Vorhersagegenauigkeit wiegen aber beide Fehler genau gleich schwer. Für die "ökonomische" Beurteilung der Qualität des Algorithmus ist also die Vorhersagegenauigkeit nicht geeignet.

In unserem Fall lohnt es sich also sehr, wenn man in der Lage ist, Unternehmen, die später zahlungsunfähig werden, bereits bei der Kreditvergabe korrekt zu identifizieren und diesen Unternehmen erst gar keinen Kredit zu geben. Wie misst man, ob der Algorithmus dazu in der Lage ist?

Wir kennen bereits die **Confusion-Matrix**. Sie zeigt genau, welche Art von Fehlern gemacht wurden. Wie sieht die Confusion-Matrix des naiven Modells aus? Wie die der Logistischen Regression? Wenn sie sich auch in der Verhersagegenauigkeit kaum unterscheiden, unterscheiden Sie sich dann vielleicht aber in der Art der Fehler die gemacht werden?

In [ ]:
# scikit-learn Funktion importieren
from sklearn.metrics import confusion_matrix

**Konfusionsmatrix des naiven Modells**:

In [ ]:
cm_naiv = confusion_matrix(y, y_pred_naiv)
cm_naiv

**Interpretation**:
- der naive Schätzer sagt für alle Unternehmen "non-Default" voraus, d.h. wir haben in der Confusion-Matrix nur Werte in der ersten Spalte.
- Alle 498 Unternehmen, die zahlungsunfähig wurden, wurden falsch klassifiziert. Wir haben also hier nur (die sehr teuren) Fehler vom Typ "Kredit vergeben, Unternehmen zahlt nicht zurück".

**Konfusionsmatrix des logistischen Regressionsmodells (bei Threshold $p_{thr} = 0.5$)**:

In [ ]:
# Voraussagen der Logistischen Regression bei Threshold 0.5
y_pred = model.predict(X)

In [ ]:
# Confusion Matrix der Logistischen Regression mit Threshold 0.5
cm_logreg = confusion_matrix(y, y_pred)
cm_logreg

**Interpretation**:
- Der Logistischen Regression gelang es, die kostspieligen Feher vom Typ "Kredit gegeben, nicht zurückgezahlt" zu reduzieren. Damit konnte viel Schaden abgewandt werden. Von den 498 zahlungsunfähigen Unternehmen konnten immerhin 103 korrekt als solche vorausgesagt werden. (Diese 103 Unternehmen würden bei der Kreditvergabe abgelehnt.)
- Allerdings macht die Logistische Regression nun auch einige Fehler vom Typ "Kredit verweigert, wäre aber zurückgezahlt worden". Dadurch entgehen dem Unternehmen Zinseinnahmen. 40 "guten" Unternehmen wurde unnötigerweise ein Kredit verweigert.
- Welcher der beiden Effekte nun überwiegt, hängt vom konkreten Geschäftsmodell ab.
- Typischerweise ist es in der Praxis so, dass der Verlust durch einen fälschlich vergebenen Kredit gerne mal so schwer wiegt, wie der Verdienstausfall durch 10 Kredite, die man unnötigerweise abgelehnt hat.

### Aus der Confusion-Matrix abgeleitete Performance Masse: True Positive Rate und False Positive Rate

Wir bestimmen nun mit Hilfe der Confusion-Matrix zwei weitere Performance Masse: die **True Positive Rate** (**TPR**) und die **False Positive Rate** (**FPR**). Diese beiden Performance-Masse zeigen genau, welche der obigen beiden Arten von Fehlern bei der Klassifikation gemacht wurden.

Beachten Sie, in unserem Beispiel bedeutet "positive": zur Klasse 1 zugehörig, d.h. zahlungsunfähig werden.

Wir verwenden im Folgenden die oben berechnete Confusion-Matrix des Logistischen Modells um die True Positive Rate Schritt für Schritt zu bestimmen.

Zunächst die **Positives**, also die Anzahl der Unternehmen, die tatsächlich zahlungsunfähig wurden.

In [ ]:
# das ist die Summe der Werte in der 2. Zeile der Confusion Matrix
n_positives = cm_logreg[1].sum()   # Beachten Sie: die zweite Zeile hat die Index-Position 1
n_positives

Schliesslich die **True Positives**, also die Anzahl der Unternehmen, für die der Algorithmus **korrekterweise**  die Zahlungsunfähigkeit vorausgesagt hat.

In [ ]:
# Das ist der Wert rechts unten in der Confusion-Matrix
true_positives = cm_logreg[1,1]
true_positives

Der Anteil der tatsächlichen Defaults (*Positives*), die vom Algorithmus korrekterweise auch als Defaults vorausgesagt wurden (*True Positives*), ist die **True Positive Rate**:

True Positive Rate = True Positives / Positives

In [ ]:
# Die True Positive Rate ist der Quotient:
true_positives / n_positives

Dieser Anteil der wahren Defaults, die vom Algorithmus korrekterweise als solche erkannt wurden, nennt sich die **True Positive Rate** oder **TPR**. Im Kontext des Machine Learning wird die TPR manchmal auch **Recall-Score** genannt, so auch in scikit-learn. Scikit-learn stellt zur Berechnung der True Positive Rate die Fuktion *recall_score()* zur Verfügung. Dieser Funktion müssen als Parameter die wahre Klassenzugehörigkeit und die vorausgesagte Klassenzugehörigkeit übergeben werden.

In [ ]:
# Die scikit-learn Funktion importieren
from sklearn.metrics import recall_score

In [ ]:
# Die True Positive Rate berechnen 
tpr = recall_score(y, y_pred)
tpr

Die Logistische Regression war also in der Lage, etliche (gut 20%) der späteren Zahlungsausfälle korrekt vorauszusagen. Ein gutes Modell sollte eine **möglichst hohe** *True Positive Rate* haben. Das 'perfekte' Modell hätte eine TPR von 1!

**Bemerkungen**:

* Das naive Modell hat eine TPR von 0, da alle Unternehmen als non-Defaults klassifiziert wurden, also gar keine *positives* erkannt wurden.
* Ein (naives) Modell, das **alle** Unternehmen als Defaults klassifizieren würde, hätte eine TPR von 1.

Leider ist es üblicherweise so, dass je besser ein Modell in der Lage ist, die wahren Defaults korrekt als solche zu klassifizieren, desto öfter macht es den Fehler, in Wahrheit 'gesunde' Unternehmen (non-Deafults) fälschlicherweise auch als Defaults zu klassifizieren. Der Anteil der 'gesunden' Unternehmen (non-Defaults), die **fälschlicherweise** vom Modell als Defaults klassifiziert werden, heisst **False Positive Rate** oder **FPR**. Für die *False Positive Rate* gibt es keine eigene Funktion in scikit-learn, aber sie kann natürlich aus der Confusion-Matrix berechnet werden.

Zunächst die **Negatives**, also die Anzahl der Unternehmen, die tatsächlich **nicht** zahlungsunfähig wurden.

In [ ]:
# Das ist die Summe der Werte in der ersten Zeile der Confusion-Matrix
n_negatives = cm_logreg[0].sum()  # Beachten Sie: die erste Zeile hat die Index-Position 0 
n_negatives

Schliesslich die **False Positives**, also die Anzahl der Unternehmen, für die der Algorithmus **fälschlicherweise** die Zahlungsunfähigkeit vorausgesagt hat.

In [ ]:
# False Positives: Anzahl Unternehmen, die tatsächlich non-Defaults sind, aber fälschlicherweise als Defaults vorausgesagt wurden
# Das ist der Wert in der Confusion-Matrix rechts oben
false_positives = cm_logreg[0,1]
false_positives

Der Anteil der "gesunden" Unternehmen (*Negatives*), die vom Algorithmus fälschlicherweise als Defaults vorausgesagt wurden (*False Positives*), ist die **False Positive Rate**:

False Positive Rate = False Positives / Negatives

In [ ]:
# False Positive Rate
fpr = false_positives/ n_negatives
fpr

Die Logistische Regression hätte also bei einigen (bei knapp 1%) 'gesunden' Unternehmen zu unrecht zur Ablehnung des Kreditantrags geführt, da das Modell fälschlicherweise einen Zahlungsausfall vorausgesagt hätte. Das ist natürlich nicht gut. Ein gutes Modell sollte eine möglichst geringe *False Positive Rate* haben. Das perfekte Modell hätte eine FPR von 0!

**Bermekungen:**

* Die FPR des naiven Modells (alle Unternehmen sind non-Defaults) wäre 0. Super!
* Die FPR des naiven Modells (alle Unternehmen sind Defaults) wäre 1. Super-Schlecht!

### Zusammenfassung:

Wir haben gesehen, dass ein **gutes Modell** sowohl eine **möglichst hohe TPR**, als auch eine **möglichst niedrige FPR** haben sollte. Wenn wir die Qualität zweier Modelle vergleichen wollen, genügt es daher nicht, **nur** die TPR oder **nur** die FPR anzuschauen, sondern beide Werte sind gemeinsam zu betrachten und gegeneinander abzuwägen.

Wie der Ausgleich zwischen hoher TPR und tiefer FPR zu finden ist, hängt vom Anwendungsfall ab: z.B. wie schlimm ist es, einen 'guten Kunden' fälschlicherweise abzulehnen und wie nützlich ist es einen 'schlechten Kunden' richtigerweise abzulehnen? 

**Exkurs: Beispiel Kreditgeschäft**

Nemen Sie an, die Bank verliert bei der Vergabe eines Kredits an einen Kunden, der später nicht in der Lage ist, diesen zurück zu zahlen, den gesamten Kreditbetrag von CHF 100'000. Lehnt die Bank einen Kunden ab, der eigentlich in der Lage wäre den Kredit (samt Zinsen) zurück zu zahlen, so verliert sie CHF 2'500 (Zins-)einkommen.

Ein Algorithmus habe 50 Zahlungsausfälle korrekt vorausgesagt, allerdings 1390 (gute) Kunden fälschlicherweise abgelehnt. Was wäre der Nettoeffekt (in CHF) dieses Algorithmus für die Bank (verglichen mit dem "Base Case", dass alle Anträge angenommen würden)?

In [ ]:
# Vermiedene Verluste (nicht zurückbezahlte Kredite):
verm_verl = 50* 100000
verm_verl

In [ ]:
# Verloren gegangenes Einkommen (ablehnen "guter" Kunden)
verl_eink = 1390*2500
verl_eink

In [ ]:
# Nettoeffekt: Vermiedene Verluste wirken Positiv, entgangenes Einkommen negativ
verm_verl - verl_eink

Obwohl es also auf den ersten Blick desaströs aussieht, 1390 (gute) Kunden abzulehnen, hat sich der Einsatz des Algorithmus finanziell doch gelohnt, da es gelang die sehr grossen Verluste durch die 50 Ausfälle durch "schlechte" Kunden zu vermeiden.

**Exkurs Ende**

## Die Receiver Operating Characteristic - ROC

Die Logistische Regression so wie sie oben implementiert wurde war in der Lage, etliche Defaults korrekt vorherzusagen (recht hohe TPR), wobei sie aber auch ein paar Fehler machte (zwar kleine FPR, aber nicht 0). Man hätte die Klassifizierung aber auch anders machen können - mit genau demselben Modell! Die **Logistische Regression** selbst **modelliert** ja zunächst nur die **Ausfallwahrscheinlichkeiten**. Erst durch Einführen eines **Thresholds** $p_{thr}$ haben wir die Grenze zwischen 'Default' (Kredit Ablehnen) und non-Default (Kredit Annehmen) festgelegt. Oben wurde diese Klassifikation mit $p_{thr} = 0.5$ durchgeführt.

Ein **anderer Threshhold** hätte zu einer **anderen Klassifikation** der Unternehmen geführt. Vielleicht hätte man durch einen anderen Threshold sogar eine bessere Performance erzielen können?

**Bemerkungen:**:
* Je höher der Threshold, desto tiefer die FPR (gut!) aber auch die TPR (schlecht!).
* Je niedriger der Threshold, desto höher die FPR (schlecht!) aber auch die TPR (gut!).
* Der Einfluss der Veränderung des thresholds auf dei Vorhersagegenauigkeit ist unklar, sie kann steigen oder fallen oder (fast) unverändert bleiben.

**Beispiel**: die Confusion-Matrix, Accuracy, TPR und FPR unseres Logistischen Modells *model* für verschiedene Thresholds.

In [ ]:
# Die verhergesagten Ausfallwahrscheinlichkeiten
y_pred_proba = model.predict_proba(X)[:,1]
y_pred_proba

In [ ]:
# Die Klassifikation bei gegebenem Threshold, z.b. p_thr = 0.25
y_pred_thr = (y_pred_proba > 0.25)*1
y_pred_thr

In [ ]:
# Die resultierende Konfusionsmatrix
cm_thr = confusion_matrix(y, y_pred_thr)
cm_thr

In [ ]:
# Die Performance Masse Accuracy, TPR und FPR
print('Accuracy:', (cm_thr[0,0] + cm_thr[1,1]) / cm_thr.sum())
print('TPR:', cm_thr[1,1]/cm_thr[1].sum())
print('FPR:', cm_thr[0,1]/cm_thr[0].sum())

Ein Diagramm, das den **Verlauf von TPR und FPR für **verschiedene Thresholds** zeigt, ist die **Receiver Operating Characteristic**, auch **ROC-Kurve** genannt.

Scikit-learn kennt hier die Funktion *roc_curve()*. Sie benötigt als Input die **wahre Klassenzugehörigkeit** und die **vorausgesagte Wahrscheinlichkeit**, zur Klasse 1 (Default) zu gehören.

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
# roc_curve() liefert drei Listen: FPR, TPR und verwendete Thresholds
fpr, tpr, thresholds = roc_curve(y, y_pred_proba)

In [ ]:
thresholds

In [ ]:
# TPR und FPR in Abhängigkeit des Thresholds
plt.title('TPR (grün) und FPR (rot) in Abhängigkeit des Thresholds')
plt.xlabel('Threshold')
plt.ylabel('Rate')
plt.plot(thresholds[1:], fpr[1:], color = 'red') # Für eine schönere Darstellung lassen wir den ersten Wert (Indexposition 0) weg
plt.plot(thresholds[1:], tpr[1:], color = 'green') # Für eine schönere Darstellung lassen wir den ersten Wert (Indexposition 0) weg

Die **ROC-Kurve** stellt nun die **True Positive Rate als Funktion der False Positive Rate** dar.

In [ ]:
# ROC-Kurve
plt.title('ROC Kurve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot(fpr, tpr)

### Bemerkungen:

* Der naive Schätzer "alle sind non-Defaults" entspricht $p_{thr} = 1$ und hat die FPR = 0 und TPR = 0
* Der naive Schätzer "alle sind Defaults" entspricht $p_{thr} = 0$ und hat die FPR = 1 und TPR = 1
* Erhöht man den Threshold $p_{thr}$ von 0 auf 1, geht man im Diagramm der Linie entlang von oben rechts nach unten links.

**Wichtige Bemerkung**:

Welches der optimale Threshold ist, hängt vom einzelnen Anwendungsfall ab. Aber grundsätzlich gilt: Je weiter die Kurve nach oben durchgebogen ist, desto besser ist das Modell. Warum?

Ein 'einfaches' Mass das reflektiert, wie stark die Kurve nach oben durchgebogen ist, ist die sogenannte **Area Under the Curve (AUC)**. Sie misst die gesamte Fläche unter der Kurve. In scikit-learn gibt es dazu die Funktion *roc_auc_score()*. Die Argumente dieser Funktion sind dieselben wie die der Funktion *roc_curve()*, nämlich die wahre Klassenzugehörigkeit und die vorhergesagte Wahrscheinlichkeit.

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y, y_pred_proba)

Viel schneller geht die Analyse des Verhaltens des Modells bei verschiedenen Thresholds mit Hilfe der in scikit-learn vordefinierten Klasse *RocCurveDisplay()* und den in dieser Klasse definierten Methoden *from_predictions()* und *from_estimator()*.

Als Parameter benötigt die Methode *from_predictions()* die wahre Klassenzugehörigkeit und die vom Modell vorausgesagte Wahrscheinlichket der Zugehörigkeit zur Klasse 1.

In [ ]:
from sklearn.metrics import RocCurveDisplay

In [ ]:
# aus den Vorhersagen des Modells
RocCurveDisplay.from_predictions(y, y_pred_proba)

Die Methode *from_estimator()* benötigt als Parameter das trainierte Modell, sowie die Features und Labels der zu verwendenden Daten.

In [ ]:
# Alternativ: direkt mit Hilfe des trainierten Modells
RocCurveDisplay.from_estimator(model, X, y)